In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [ ]:
# Function to Scrape Yelp Reviews

def scrape_yelp_pages(url):
    # Set up Chrome driver service
    service = webdriver.chrome.service.Service('/Users/JohnnyBlaze/drivers/chromedriver')  
    service.start()

    # Set up Chrome options
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  

    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome(service=service, options=options)

    # Navigate to the webpage
    driver.get(url)

    # Wait for the page to load and for the first page number element to be visible
    page_number_element = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//div[@aria-label="Page: 1"]'))
    )

    reviews = []
    star_ratings = []
    dates = []

    while True:
        # Process the current page here
        review_elements = driver.find_elements(By.XPATH, '//span[@class=" raw__09f24__T4Ezm" and @lang="en"]')
        star_rating_elements = driver.find_elements(By.XPATH, '//div[contains(@aria-label, "star rating")]')
        date_elements = driver.find_elements(By.XPATH, '//span[@class=" css-chan6m"]')

        for review_element, star_rating_element, date_element in zip(review_elements, star_rating_elements, date_elements):
            review_text = review_element.text
            star_rating = star_rating_element.get_attribute('aria-label')
            date = date_element.text

            # Ignore dates with the words 'Photos', 'years', 'Photo', and 'year'
            if 'Photos' not in date and 'years' not in date and 'Photo' not in date and 'year' not in date:
                reviews.append(review_text)
                star_ratings.append(star_rating)
                dates.append(date)

        try:
            # Click on the next page
            next_page_arrow = driver.find_element(By.XPATH, '//span[@class="icon--24-chevron-right-v2 navigation-button-icon__09f24__Bmrde css-1kq79li"]')
            next_page_arrow.click()

            # Wait for a brief moment for the new review elements to load
            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '//span[@class=" raw__09f24__T4Ezm" and @lang="en"]')))

        except:
            # Break the loop when there are no more pages to click
            break

    # Close the browser window
    driver.quit()

    # Create a DataFrame with the scraped data
    data = pd.DataFrame({'Date': dates, 'Review': reviews, 'Star Rating': star_ratings})

    return data

In [ ]:
# Filippi's Pizza

pizza_df = scrape_yelp_pages('https://www.yelp.com/biz/filippis-pizza-grotto-mission-valley-san-diego?osq=pizza')

pizza_df.head()

In [ ]:
# To CSV

csv_file_path = '/Users/JohnnyBlaze/filippi.csv'

pizza_df.to_csv(csv_file_path, index=False)

In [ ]:
# Tacos El G

taco_df = scrape_yelp_pages('https://www.yelp.com/biz/tacos-el-g-national-city-7?osq=Tacos')

taco_df.head()

In [ ]:
# To CSV

csv_file_path = '/Users/JohnnyBlaze/taco_el_g.csv'

taco_df.to_csv(csv_file_path, index=False)